# 💬🌽 Semantic Kernel Agents V1 Time Capsule

This is going to change in Semantic Kernel 1.10, but it's been swimming around in my head since SK 1.0 to better understand. It walks through how to: 1/ give the agent an SK Plugin, 2/ use the OAI Assistants retrieval option with files, and 3/ use the code interpreter to create a PowerPoint presentation file.

## 📦 Get the right packages together

In [4]:
#r "nuget: Microsoft.SemanticKernel, 1.9"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.9-alpha"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.9-alpha"

Installed Packages Microsoft.SemanticKernel, 1.9.0 Microsoft.SemanticKernel.Experimental.Agents, 1.9.0-alpha Microsoft.SemanticKernel.Plugins.Core, 1.9.0-alpha

## 🔥 Fire up a 🌽 kernel and 🥸 agentbuilder

In [7]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Experimental.Agents;
using Microsoft.Extensions.Logging;
using System.ComponentModel;
using System.Net.Http;
using System.Net.Http.Headers;

#pragma warning disable SKEXP0010, SKEXP0101, CS0103

using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;
static List<IAgent> _agents = [];
static string[] _fileIds = [];
    
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

model = "gpt-4-turbo";

kernel = !useAzureOpenAI ?
    Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(model, apiKey, orgId)
        .AddOpenAIFiles(apiKey)
        .Build() :
    Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .AddAzureOpenAIFiles(azureEndpoint, apiKey)
        .Build();

public AgentBuilder CreateAgentBuilder()
{
    var builder = new AgentBuilder();

    return
        !useAzureOpenAI ?
            builder.WithOpenAIChatCompletion(model, apiKey) :
            builder.WithAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
}

public static IAgent Track(IAgent agent)
{
    _agents.Add(agent);

    return agent;
}

public static IAgent GetAgent(string name)
{
    return _agents.FirstOrDefault(a => a.Name == name);
}

public async Task<string> UploadAgentsRelatedFile(string filename)
{
    FileStream fileStream = new FileStream(filename, FileMode.Open);
    var fileService = kernel.GetRequiredService<OpenAIFileService>();

    var result =
        await fileService.UploadContentAsync(
            new BinaryContent(() => Task.FromResult((Stream)fileStream)),
            new OpenAIFileUploadExecutionSettings(filename, OpenAIFilePurpose.Assistants));   
    _fileIds = _fileIds.Append(result.Id).ToArray();
    return result.Id;
}

public async Task CleanupAgents()
{
    var fileService = kernel.GetRequiredService<OpenAIFileService>();
    await Task.WhenAll(_agents.Select(a => a.DeleteAsync()));
    await Task.WhenAll(_fileIds.Select(fileId => fileService.DeleteFileAsync(fileId)));

    _agents = new List<IAgent>();
    _fileIds = new string[] {};
}

## 🪚 Build a simple agent 👮 router

In [8]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103

public static string InfoAboutAgents()
{
    var agentsInfo = _agents.Select(a => $"Name '{a.Name}': Description: {a.Description}\n").ToList();
    agentsInfo.Insert(0, "Assistants available with their names and descriptions:\n");
    return agentsInfo.Aggregate((a, b) => a + b);
}

public async Task<string> GetAgentForQuestion(string question)
{
    var aboutAgents = InfoAboutAgents();
    var result = await kernel.InvokePromptAsync($"{aboutAgents}\nGive just the assistant's name that can answer the question: '{question}''. If you don't know the answer, respond 'Unknown.'");
    return result.ToString();
}

## 🔌 Plugins can be handed over to any of your agents

In [9]:
public class TimeInformationPlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

var timePlugin = KernelPluginFactory.CreateFromType<TimeInformationPlugin>();


## 🐣 Hatch a few 🥸 agents and provide 📁 files for grounding

In [10]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103

var fileId = await UploadAgentsRelatedFile("travelinfo.txt");

var retrievalAgent = Track(await CreateAgentBuilder()
    .WithName("Retrieval")
    .WithDescription("Really good at finding information about people")
    .WithInstructions("Respond succinctly with the information requested.")
    .WithRetrieval()
    .BuildAsync());

await retrievalAgent.AddFileAsync(fileId);

var timeAgent = Track(await CreateAgentBuilder()
    .WithName("Timeman")
    .WithDescription("Knows what time it is")
    .WithInstructions("Respond succinctly with the information requested.")
    .WithPlugin(timePlugin)
    .BuildAsync());

async Task InvokeAgentAsync(IAgent agent, string question)
{
    await foreach (var message in agent.InvokeAsync(question, null, _fileIds))
    {
        string content = message.Content;
        foreach (var annotation in message.Annotations)
        {
            content = content.Replace(annotation.Label, string.Empty, StringComparison.Ordinal);
        }

        Console.WriteLine($"# {message.Role}: {content}");

        if (message.Annotations.Count > 0)
        {
            var fileService = kernel.GetRequiredService<OpenAIFileService>();
            Console.WriteLine("\nAnnotation related files:");
            foreach (var annotation in message.Annotations)
            {
                Console.WriteLine($"* {annotation.FileId}");
            }
        }
    }
    Console.WriteLine();
}

## 🧪 Test the agent 👮 router to see if it works

In [11]:
var name = await GetAgentForQuestion("What time is it?");
Console.WriteLine($"Agent name: {name}");

Agent name: Timeman


I don't use this code but I thought it's useful to show you how you can get the agents to talk with each other based upon their different expertise. You just ask for the agent that can resolve a specific issue and route the issue accordingly.

## 💬 Chat with the 🥸 agent of your choice 

When you run the code below, the notebook is asking a question at the very to of the screen. 👆 Look up at the top of your VS Code window!

![](assets/lookup.png)

Get it? When you run the code below you need to 👆 go to the top of this window to enter your input. I always forget that ...

> ⚠️ _Be sure to clean up your agent work by going to the cell just below this one_

In [12]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103

try
{
    // sit in a loop and talk to the agent and when you enter /bye then exit
    var myq = await InteractiveKernel.GetInputAsync("Your turn: ");

    while (myq != "/bye")
    {
        await InvokeAgentAsync(retrievalAgent, myq);
        myq = await InteractiveKernel.GetInputAsync("Your turn: ");
    }
}
finally
{
    Console.WriteLine("⚠️ Completed. Be sure to clean up your agents.");
}

# user: Who is Fujii Kaze?
# assistant: I will need to access the contents of the file you uploaded to provide information on Fujii Kaze from within the document. Please give me a moment to retrieve the information.
# assistant: The document you provided appears to be related to travel information and does not contain any details regarding Fujii Kaze. If you have another document or question about Fujii Kaze, please provide additional information or files.

⚠️ Completed. Be sure to clean up your agents.


## 🧽 Clean up your 🥸 agents and any 📁 files you uploaded

Various items will stay inside your account unless you consciously purge them. Otherwise you'll have to manually delete them 🙀 ...

In [13]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103
// print the current time in the current timezone
Console.WriteLine($"Current time: {DateTime.Now}");

Console.WriteLine("🧽 Cleaning up...");
await CleanupAgents();
Console.WriteLine("👋 Done. Bye!");

Current time: 4/29/2024 4:59:29 AM
🧽 Cleaning up...
👋 Done. Bye!


## ⛰️ Advanced: Let the agent run 🐍 Python code to write a file

In [20]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103

var fileId = await UploadAgentsRelatedFile("travelinfo.txt");

var codeInterpreterPlusRetrievalAgent = Track(await CreateAgentBuilder()
    .WithName("Crafter")
    .WithDescription("Really good at crafting code, finding info about people, and creating powerpoints.")
    .WithInstructions("Write only code to solve the given problem without comment.")
    .WithRetrieval()
    .WithCodeInterpreter()
    .BuildAsync());

await codeInterpreterPlusRetrievalAgent.AddFileAsync(fileId);

async Task InvokeAgentAsyncSaveToFile(IAgent agent, string question, string? outputFileName = null)
{
    await foreach (var message in agent.InvokeAsync(question, null, _fileIds))
    {
        string content = message.Content;
        foreach (var annotation in message.Annotations)
        {
            content = content.Replace(annotation.Label, string.Empty, StringComparison.Ordinal);
        }

        Console.WriteLine($"# {message.Role}: {content}");

        if (message.Annotations.Count > 0)
        {
            var fileService = kernel.GetRequiredService<OpenAIFileService>();
            Console.WriteLine("\nAnnotation related files:");
            foreach (var annotation in message.Annotations)
            {
                Console.WriteLine($"* {annotation.FileId}");
                if (outputFileName != null) {
                    // Download file if given a name
                    // Note that if there are multiple files then this will not work the way you want
                    using (var httpClient = new HttpClient())
                    {
                        httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", apiKey);
                        var response = await httpClient.GetAsync($"https://api.openai.com/v1/files/{annotation.FileId}/content");
                        var fileContent = await response.Content.ReadAsByteArrayAsync();

                        // Write file to local directory
                        Console.WriteLine($"Writing file to {outputFileName}");
                        await File.WriteAllBytesAsync(outputFileName, fileContent);
                        Console.WriteLine("File written.");
                    }
                }          
            }
        }
    }
    Console.WriteLine();
}

## 🚵 Run the 🥸 agent with 🧑‍💻 code interpreter

This took me about 50 seconds to complete. So be a little patient while crossing your fingers ...

In [21]:
await InvokeAgentAsyncSaveToFile(
    codeInterpreterPlusRetrievalAgent, 
    "Create a powerpoint file that documents Sam's travel plans.", 
    "sam.pptx");
Console.WriteLine("👋 Done. Bye!");

# user: Create a powerpoint file that documents Sam's travel plans.
# assistant: Based on the detailed travel plan in the document, it's clear that Sam Albert Marks is scheduled for a round trip from Seattle to Raleigh-Durham for a stay of 2 days. Here is the summary of the travel plan:

### Travel Summary:
- **Air Travel:** Sam is booked on Alaska Airlines.
  - Outbound Flight: Flight AS 572 from Seattle (SEA) to Durham (RDU) on Monday, September 11, 2023. Departure at 10:05 AM and arrival at 6:15 PM.
  - Return Flight: Flight AS 491 from Durham (RDU) to Seattle (SEA) on Wednesday, September 13, 2023. Departure at 7:15 PM and arrival at 9:59 PM.
- **Accommodation:** Sam is staying at The Westin Raleigh Durham.
  - Check-in: Monday, September 11, 2023
  - Check-out: Wednesday, September 13, 2023
  - Address: 3931 Macaw Street, Raleigh, NC, 27617, US
  - Phone: (1) 919-224-1400 
  - Room Rate: USD 280.00 per night, may be subject to local taxes and service charges.

### Additional Detai

## 🧽 Clean up your 🥸 agents and any 📁 files you uploaded

This is missing the file you created ... I'll let you figure out how to do that ;-).

In [22]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103
// print the current time in the current timezone
Console.WriteLine($"Current time: {DateTime.Now}");

Console.WriteLine("🧽 Cleaning up...");
await CleanupAgents();
Console.WriteLine("👋 Done. Bye!");

Current time: 4/29/2024 5:15:51 AM
🧽 Cleaning up...
👋 Done. Bye!
